In [5]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import datetime
from math import ceil
from lxml import etree

In [6]:
url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [8]:
def obtain_parse_wiki_snp500():
    #stores the current time
    now=datetime.datetime.utcnow()
    page=urlopen(url)
    htmlparser=etree.HTMLParser()
    tree=etree.parse(page,htmlparser)
    symbolslist=tree.xpath('//table[1]/tr')[1:]
    
    symbols=[]
    for symbol in symbolslist:
    
        tds=symbol.getchildren()
        sd={'ticker'}:tds[0].getchildren()[0].text,
            'name':tds[1].getchildren()[0].text,
            'sector':tds[3].text}
symbols.append((sd['ticker'],'stock',sd['name'],'USD',now,now))

    Return symbols

SyntaxError: invalid syntax (<ipython-input-8-23fd600aff3d>, line 13)

In [9]:
import sqlite3
conn=sqlite3.connect('example.db')
c=conn.cursor()

In [12]:
#Create table
c.execute('''CREATE TABLE STOCKORDERS
             (date text, trans text, symbol text, qty real, price real)''')

#insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

#Save (commit) the changes
conn.commit()

#We can also close the connection if we are done with it.
#just be sure any changes have been committed or they will be lost.
conn.close()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 71))



OperationalError: table STOCKORDERS already exists

In [15]:
import sqlite3
conn = sqlite3.connect('example.db')
c = conn.cursor()
t=('RHAT',)

c.execute('SELECT * FROM STOCKORDERS WHERE symbol=?',t)
print(c.fetchone())

None


In [16]:
#Larger example that inserts many redords at a time
purchases = [('2006-03-28', 'BUY','IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)',purchases)

for row in c.execute('SELECT * FROM stocks ORDER BY price'):
    print(row)

OperationalError: no such table: stocks

In [22]:
#!/usr/bin/env python

import urllib2
import pytz

import pandas as pd
from pandas_datareader import data, wb

from bs4 import BeautifulSoup
from datetime import datetime



SITE = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
START = datetime(1900, 1, 1, 0, 0, 0, 0, pytz.utc)
END = datetime.today().utcnow()


def scrape_list(site):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(site, headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)

    table = soup.find('table', {'class': 'wikitable sortable'})
    sector_tickers = dict()
    for row in table.findAll('tr'):
        col = row.findAll('td')
        if len(col) > 0:
            sector = str(col[3].string.strip()).lower().replace(' ', '_')
            ticker = str(col[0].string.strip())
            if sector not in sector_tickers:
                sector_tickers[sector] = list()
            sector_tickers[sector].append(ticker)
    return sector_tickers


def download_ohlc(sector_tickers, start, end):
    sector_ohlc = {}
    for sector, tickers in sector_tickers.iteritems():
        print 'Downloading data from Yahoo for %s sector' % sector
        data = DataReader(tickers, 'yahoo', start, end)
        for item in ['Open', 'High', 'Low']:
            data[item] = data[item] * data['Adj Close'] / data['Close']
        data.rename(items={'Open': 'open', 'High': 'high', 'Low': 'low',
                           'Adj Close': 'close', 'Volume': 'volume'},
                    inplace=True)
        data.drop(['Close'], inplace=True)
        sector_ohlc[sector] = data
    print 'Finished downloading data'
    return sector_ohlc


def store_HDF5(sector_ohlc, path):
    with pd.get_store(path) as store:
        for sector, ohlc in sector_ohlc.iteritems():
            store[sector] = ohlc


def get_snp500():
    sector_tickers = scrape_list(SITE)
    sector_ohlc = download_ohlc(sector_tickers, START, END)
    store_HDF5(sector_ohlc, 'snp500.h5')


if __name__ == '__main__':
    get_snp500()

NameError: global name 'DataReader' is not defined

** Below is the code to scrap and build a stock database in R**

In [2]:

# Quandl package must be installed
library(Quandl)
 
# Get your API key from quandl.com
quandl_api = "MYAPIKEY"
 
# Add the key to the Quandl keychain
Quandl.api_key(quandl_api)
 
quandl_get <-
function(sym, start_date = "2017-01-01") {
    require(devtools)
    require(Quandl)
    # create a vector with all lines
    tryCatch(Quandl(c(
        paste0("WIKI/", sym, ".8"),  #  Adj. Open
        paste0("WIKI/", sym, ".9"),  # Adj. High
        paste0("WIKI/", sym, ".10"), # Adj. Low
        paste0("WIKI/", sym, ".11"), # Adj. Close
        paste0("WIKI/", sym, ".12")), # Adj. Volume
        start_date = start_date,
        type = "zoo"
        ))
}

SyntaxError: invalid syntax (<ipython-input-2-d540eb28cf90>, line 11)

In [ ]:
# Load list of symbols (Updated May 2017)
SYM <- as.character( read.csv('http://trading.chrisconlan.com/SPstocks_current.csv', 
                              stringsAsFactors = FALSE, header = FALSE)[,1] )
 
 
# Hold stock data and vector of invalid requests
DATA <- list()
INVALID <- c()
 
# Attempt to fetch each symbol
for(sym in SYM){
  google_out <- google(sym)
   
  if(!is.null(google_out)) {
    DATA[[sym]] <- google_out
  } else {
    INVALID <- c(INVALID, sym)
  }
}
 
# Overwrite with only valid symbols
SYM <- names(DATA)
 
# Remove iteration variables
rm(google_out, sym)
 
cat("Successfully download", length(DATA), "symbols.")
cat(length(INVALID), "invalid symbols requested.\n", paste(INVALID, collapse = "\n\t"))
cat("We now have a list of data frames of each symbol.")
cat("e.g. access MMM price history with DATA[['MMM']]")